In [ ]:
#Importing Packages
#Basic
import numpy as np
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
from collections import deque
import random
import subprocess


#Networks
import networkx as ntx

#Visualizing Tool
from graph_tool.all import *
import graph_tool.draw as gd
import graph_tool.centrality as gc
import cairo
import inspect
from graph_tool.draw.graphviz_draw import *
from graph_tool.generation import *

#Fractal Dimension
import json
from scipy.stats import linregress

In [ ]:
#Generate output file name
def png_namegen(dtype='FB', gtype='ESpS', BAedge=None, wtype=None, eps=None, cutoff = None):
    name = './graphs/'+gtype+'/'+('_'.join([dtype,gtype]))
    if BAedge: name = '_'.join([name,'BA',str(BAedge)])
    if wtype: name = '_'.join([name,'wtype',wtype])
    if eps: name = '_'.join([name,'eps','{:.1e}'.format(eps)])
    if cutoff: name = '_'.join([name,'cut',str(cutoff)])
    name += '.png'
    return name
def hist_namegen(dtype='FB', htype='degree', BAedge=None, st=None, wtype=None, breaks=None, rang=None):
    name = './histograms/'+htype+'/'+('_'.join([dtype,htype]))
    if BAedge: name = '_'.join([name,'BA',str(BAedge)])
    if st: name = '_'.join([name,'st',str(st)])
    if wtype: name = '_'.join([name,'wtype',wtype])
    if breaks: name = '_'.join([name,'breaks',str(breaks)])
    if rang: name = '_'.join([name,'range','{:.1e}'.format(rang)])
    name += '.png'
    return name
def adjlist_namegen(dtype='FB', gtype='ESpS', BAedge=None, wtype=None, eps=None, cutoff = None):
    name = './AdjList/'+gtype+'/'+('_'.join([dtype,gtype]))
    if BAedge: name = '_'.join([name,'BA',str(BAedge)])
    if wtype: name = '_'.join([name,'wtype',wtype])
    if eps: name = '_'.join([name,'eps','{:.1e}'.format(eps)])
    if cutoff: name = '_'.join([name,'cut',str(cutoff)])
    name += '.txt'
    return name

In [ ]:
#Dehash and Get Adjacency List
def dehash_and_listize(G, undir = True):
    hasher = []
    for e in G:
        hasher.append(e[0])
        hasher.append(e[1])
    hasher = list(set(hasher))
    print(hasher[:50])
    dehasher = {}
    for i, h in enumerate(hasher):
        dehasher[h] = i 
    
    n = len(hasher)
    adj_G = [[] for _ in range(n)]
    for e in G:
        if e[0] == e[1]: continue
        adj_G[dehasher[e[0]]].append(dehasher[e[1]])
        if not undir:
            adj_G[dehasher[e[1]]].append(dehasher[e[0]])
    #print(type(G))
    return hasher, adj_G


#Get Datasets
FB = np.loadtxt(fname = adjlist_namegen(dtype='FB', gtype='Raw', BAedge=None), delimiter=' ', comments='#', dtype=int)
GR = np.loadtxt(fname = adjlist_namegen(dtype='GR', gtype='Raw', BAedge=None), delimiter=' ', comments='#', dtype=int)
SWNwH = np.loadtxt(fname = adjlist_namegen(dtype='SWNwH', gtype='Raw', BAedge=None), delimiter=' ', comments='#', dtype=int)
SWNwA = np.loadtxt(fname = adjlist_namegen(dtype='SWNwA', gtype='Raw', BAedge=None), delimiter=' ', comments='#', dtype=int)
ban_1 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=1), delimiter=' ', comments='#', dtype=int)
ban_2 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=2), delimiter=' ', comments='#', dtype=int)
ban_3 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=3), delimiter=' ', comments='#', dtype=int)
ban_4 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=4), delimiter=' ', comments='#', dtype=int)
ban_5 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=5), delimiter=' ', comments='#', dtype=int)
ban_6 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=6), delimiter=' ', comments='#', dtype=int)
ban_7 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=7), delimiter=' ', comments='#', dtype=int)
ban_8 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=8), delimiter=' ', comments='#', dtype=int)
ban_9 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=9), delimiter=' ', comments='#', dtype=int)
ban_10 = np.loadtxt(fname = adjlist_namegen(dtype='BAN', gtype='Raw', BAedge=10), delimiter=' ', comments='#', dtype=int)

hasher_GR,adjlist_GR = dehash_and_listize(GR, undir = False)
hasher_FB,adjlist_FB = dehash_and_listize(FB, undir = False)
hasher_SWNwH,adj_swn_with_hub = dehash_and_listize(SWNwH, undir = False)
hasher_SWNwA,adj_swn_with_add = dehash_and_listize(SWNwA, undir = False)
hasher_ban_1,adj_ban_1 = dehash_and_listize(ban_1, undir = False)
hasher_ban_2,adj_ban_2 = dehash_and_listize(ban_2, undir = False)
hasher_ban_3,adj_ban_3 = dehash_and_listize(ban_3, undir = False)
hasher_ban_4,adj_ban_4 = dehash_and_listize(ban_4, undir = False)
hasher_ban_5,adj_ban_5 = dehash_and_listize(ban_5, undir = False)
hasher_ban_6,adj_ban_6 = dehash_and_listize(ban_6, undir = False)
hasher_ban_7,adj_ban_7 = dehash_and_listize(ban_7, undir = False)
hasher_ban_8,adj_ban_8 = dehash_and_listize(ban_8, undir = False)
hasher_ban_9,adj_ban_9 = dehash_and_listize(ban_9, undir = False)
hasher_ban_10,adj_ban_10 = dehash_and_listize(ban_10, undir = False)

In [ ]:
#Export Dataset
def export(adj_L, filename):
    n = len(adj_L)
    f = open(filename, 'w')
    for i in range(n):
        for j in adj_L[i]:
            f.write(str(i) + ' ' + str(j) + '\n')
    f.close()

In [ ]:
#Generate Erdos-Renyl Random Graph w/ |V|=n, prob=p
def erdos_renyi(n = 5000, p = 0.5):
    adj_L = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if random.random() < p:
                adj_L[i].append(j)
                adj_L[j].append(i)
    return adj_L


#Generate Modified Small-World Network w/ |V|=n+hubs, short edge:pick from 2k & prob=b, long edge = add, #hub = hubs, deg(hubs)=n*phub
def small_world_network(n = 5000, k = 20, b = 0.5, add = 0, hubs = 0, phub = 0.1):    
    # 2kb ~ mean deg, add : additional random edge 
    G = [[] for _ in range(n + hubs)]
    for i in range(n):
        for j in range(1,k+1):
            if random.random() < b:
                t = (i+j)%n
                G[i].append(t)
                G[t].append(i)
    S = set()
    while len(S) < add:
        i = random.randint(0,n-1)
        j = random.randint(i+k+1,i+n-k-1)
        j %= n
        if i > j:
            i, j = j, i
        S.add((i,j))
    #print(S)
    for elem in S:
        G[elem[0]].append(elem[1])
        G[elem[1]].append(elem[0])
    
    hubset = [i for i in range(n)]
    for i in range(hubs):
        random.shuffle(hubset)
        for j in range(int(len(hubset) * phub)):
            G[hubset[j]].append(n + i)
            G[n + i].append(hubset[j])
        hubset.append(n + i)
    return G


#Generate Scale-Free Network with Barabasi-Albert model w/ |V|=n, |E|=mn (new node: m edges)
def barabasi_albert(n = 5000, m=4):
    G = ntx.barabasi_albert_graph(n=n, m=m)
    adj = ntx.to_dict_of_lists(G)
    adj_L = [[] for _ in range(n)]
    for i in range(n):
        adj_L[i] = adj[i]
    return adj_L


#TEST ADJ
adj_swn = small_world_network(20, 3, 0.7,3, 3, 0.8)

In [ ]:
#Visualize Graph w/ AdjList
def draw_network(adj_L, pos = None, output = None):
    g = Graph(directed = False)
    n = len(adj_L)
    vlist = g.add_vertex(n)
    for i in range(n):
        for j in adj_L[i]:
            if i > j:
                continue
            e = g.add_edge(g.vertex(i), g.vertex(j))
    if not pos:
        pos = gd.sfdp_layout(g)
    graph_draw(g,pos=pos,output=output)

In [ ]:
#Calculate Efficiency w/ AdjList
def global_efficiency(adj_L):
    n = len(adj_L)
    dis = [[n + 1] * n for _ in range(n)]
    for source in range(n):
        if source % 500 == 0:
            print(source)
        D = dis[source]
        D[source] = 0
        Q = deque([])
        Q.append(source)
        while len(Q) > 0:
            f = Q.popleft()
            for u in adj_L[f]:
                if D[u] > D[f] + 1:
                    D[u] = D[f] + 1
                    Q.append(u)
    
    glob_eff = 0
    for i in range(n):
        for j in range(i+1,n):
            if dis[i][j] == n + 1:
                continue
            glob_eff += 1 / dis[i][j]
            
    glob_eff /= n * (n-1) // 2
    print('global efficiency : ', glob_eff)
    
    return glob_eff, dis


def local_efficiency(adj_L, use_log = True):
    n = len(adj_L)
    print("n = ", n)
    adj_M = [[False]*n for _ in range(n)]
    for i in range(n):
        for j in adj_L[i]:
            adj_M[i][j] = True
    tot_loc_eff = 0
    var_loc_eff = 0
    for source in range(n):
        if source % 500 == 0:
            print(source)
        B = adj_L[source]
        k = len(B)
        if k == 0:
            print("NERD")
            continue
        dis = [[k + 1] * k for _ in range(k)]
        for i in range(k):
            dq = deque([i])
            D = dis[i]
            D[i] = 0
            while len(dq) > 0:
                f = dq.popleft()
                for j in range(len(B)):
                    if not adj_M[B[i]][B[j]] or D[j] <= D[i] + 1:
                        continue
                    D[j] = D[i] + 1
                    dq.append(j)
        loc_eff = 0
        for i in range(k):
            for j in range(i+1, k):
                if dis[i][j] == k + 1:
                    continue
                loc_eff += 1 / dis[i][j]
        if k > 1:
            loc_eff /= k * (k-1) / 2
        tot_loc_eff += loc_eff
        var_loc_eff += loc_eff ** 2
    tot_loc_eff /= n
    var_loc_eff = var_loc_eff / n - tot_loc_eff ** 2
    print("Local efficiency : ", tot_loc_eff, "\nVariance : ", var_loc_eff)
    if use_log :
        f = open(log_name, 'at')
        f.write("Local efficiency : {0:.3f}\nVariance : {1:.3f}\n".
                format(tot_loc_eff, var_loc_eff))
        f.close()
    
    return tot_loc_eff, var_loc_eff

In [ ]:
#UnionFind  ///  Trim Network w.r.t. size of Conn.Comp. (Only Strictly Greater) w/ AdjList & cutoff
def uf_parent(x, par):
    if par[x] == x:
        return x
    else:
        par[x] = uf_parent(par[x], par)
        return par[x]

def uf_merge(x, y, par, sz):
    x = uf_parent(x, par)
    y = uf_parent(y, par)
    if x != y:
        par[y] = x
        sz[x] += sz[y]
        return True
    return False

def trim_network(adj_L, cutoff = 20):
    sys.setrecursionlimit(10**6)
    n = len(adj_L)
    par = [i for i in range(n)]
    sz = [1] * n
    for i in range(n):
        for j in adj_L[i]:
            uf_merge(i, j, par, sz)
    
    v_tr = []
    for i in range(n):
        if sz[uf_parent(i,par)] > cutoff:
            v_tr.append(i)
    
    hasher = [-1] * n
    for i, v in enumerate(v_tr):
        hasher[v] = i
    
    ret_L = []
    for v in v_tr:
        x = []
        for u in adj_L[v]:
            x.append(hasher[u])
        ret_L.append(x[:])
    
    return ret_L

In [ ]:
#Generate Minimal W Spanning Tree w.r.t. Large Deg -> Low Weight.   w/ AdjList & weight_type = 'plus'/'max'
#Generate Spanning Graph w.r.t. Trim Edges with Small Deg.   w/ AdjList & cutoff & weight_type = 'plus'/'max'
def elitism_spanning_tree(adj_L, weight_type='plus'):
    n = len(adj_L)
    deg = [len(adj_L[i]) for i in range(n)]
    adj_sp = [[] for _ in range(n)]
    par = [i for i in range(n)]
    sz = [1] * n
    
    weighted_edges = []
    for i in range(n):
        for j in adj_L[i]:
            if i < j: continue
            weight = 1/(deg[i] + deg[j]) if weight_type == 'plus' else 1/max(deg[i],deg[j])
            weighted_edges.append((weight, i, j))
    
    weighted_edges.sort(key = lambda t : t[0])
    print('edges : ', len(weighted_edges))
    for w in weighted_edges:
        if uf_merge(w[1], w[2], par, sz):
            #print('connecting ',w[1],w[2])
            adj_sp[w[1]].append(w[2])
            adj_sp[w[2]].append(w[1])
    return adj_sp

def elitism_spanning_subgraph(adj_L, cutoff = 0.1, weight_type='plus'):
    n = len(adj_L)
    deg = [len(adj_L[i]) for i in range(n)]
    adj_sp = [[] for _ in range(n)]
    
    weighted_edges = []
    for i in range(n):
        for j in adj_L[i]:
            if i < j: continue
            weight = 1/(deg[i] + deg[j]) if weight_type == 'plus' else 1/max(deg[i],deg[j])
            weighted_edges.append((weight, i, j))
    
    weighted_edges.sort(key = lambda t : t[0])
    print('edges : ', len(weighted_edges))
    for w in weighted_edges:
        if w[0] > cutoff:
            break
        adj_sp[w[1]].append(w[2])
        adj_sp[w[2]].append(w[1])
    return adj_sp

In [ ]:
def skeleton(adj_L, draw = False, draw_all = False, pos = None, output = None):
    n = len(adj_L)
    M = [[0] * n for _ in range(n)]
    cnt = 0
    E = []
    
    for i in range(n):
        for j in adj_L[i]:
            if i > j: continue
            M[i][j] = M[j][i] = cnt
            E.append([i,j])
            cnt += 1
    #print(M)
    Bcen = [0] * cnt
    for i in range(n):
        if i % 500 == 0:
            print('ckpt : ', i)
        dq = deque([i])
        lev = [n] * n
        bfs_ord = []
        lev[i] = 0
        Bcen_v = [0] * n
        back = [[] for _ in range(n)]
        
        while len(dq):
            f = dq.popleft()
            bfs_ord.append(f)
            for s in adj_L[f]:
                if lev[s] < lev[f] + 1: continue
                if lev[s] == lev[f] + 1:
                    back[s].append(f)
                    continue
                back[s].append(f)
                lev[s] = lev[f] + 1
                dq.append(s)
        
        #print('source = ', i)
        for f in bfs_ord[::-1]:
            Bcen_v[f] += 1
            #print('id = ', f, 'Bcen = ', Bcen_v[f], 'lev = ', lev[f], 'back = ', back[f])
            if len(back[f]) == 0: continue
            Bcen_w = Bcen_v[f] / len(back[f])
            for s in back[f]:
                #print(M[f][s], '+= ', Bcen_w)
                Bcen[M[f][s]] += Bcen_w
                Bcen_v[s] += Bcen_w
    
    skel = [(Bcen[i], i) for i in range(cnt)]

    skel.sort(key = lambda t : -t[0])
    #print(skel)
    par = [i for i in range(n)]
    sz = [1] * n
    
    adj_skel = [[] for _ in range(n)]
    adj_nskel = [[] for _ in range(n)]
    for w in skel:
        a, b = E[w[1]]
        if uf_merge(a, b, par, sz):
            adj_skel[a].append(b)
            adj_skel[b].append(a)
        else:
            adj_nskel[a].append(b)
            adj_nskel[b].append(a)
    
    if not draw:
        return adj_skel
            
    g = Graph(directed = False)
    n = len(adj_L)
    vlist = g.add_vertex(n)
    be = g.new_edge_property("double")
    
    for i in range(n):
        for j in adj_skel[i]:
            if i > j:
                continue
            e = g.add_edge(g.vertex(i), g.vertex(j))
            be[e] = Bcen[M[i][j]]
            
    if not pos:
        pos = gd.sfdp_layout(g)
    if pos == 'arf':
        pos = gd.arf_layout(g)
    if draw_all:
        for i in range(n):
            for j in adj_nskel[i]:
                if i > j: continue
                e = g.add_edge(g.vertex(i), g.vertex(j))
                be[e] = Bcen[M[i][j]]
        
    be.a /= be.a.max() / 5
    graph_draw(g,pos=pos,edge_pen_width=be,output=output)
    
    return adj_skel

In [ ]:
#Visualize Graph skeleton w/ AdjList
def draw_network_between(adj_L, pos = None, output = None):
    g = Graph(directed = False)
    n = len(adj_L)
    vlist = g.add_vertex(n)
    for i in range(n):
        for j in adj_L[i]:
            if i > j:
                continue
            e = g.add_edge(g.vertex(i), g.vertex(j))
    if not pos:
        pos = gd.sfdp_layout(g)
    
    bv, be = gc.betweenness(g)
    be.a /= be.a.max() / 5
    graph_draw(g,pos=pos,vertex_color=[1,1,1,0],vertex_fill_color=bv, edge_pen_width=be,output=output)
    
    
#Visualize Graph ESpT w/ AdjList
def draw_network_elitism(adj_L, pos = None, output = None):
    g = Graph(directed = False)
    n = len(adj_L)
    vlist = g.add_vertex(n)
    ewt = g.new_edge_property("double")
    for i in range(n):
        for j in adj_L[i]:
            if i > j:
                continue
            e = g.add_edge(g.vertex(i), g.vertex(j))
    if not pos:
        pos = gd.sfdp_layout(g)
    
    for i in range(n):
        for j in adj_L[i]:
            if i > j:
                continue
            e = g.edge(i,j)
            ewt[e] = (g.vertex(i).out_degree() + g.vertex(j).out_degree())
    #e = g.edges().next()
    #bv, be = gc.betweenness(g)
    ewt.a /= ewt.a.max() / 5
    graph_draw(g,pos=pos,vertex_color=[1,1,1,0], edge_pen_width=ewt,output=output)

In [ ]:
#Draw Degree Histogram
def get_degree_spectra(G,breaks=None,st=0, output=None):
    deg = {}
    for p in G:
        #print(p[0], p[1])
        if deg.get(p[0]) is None:
            deg[p[0]] = 1
        if deg.get(p[1]) is None:
            deg[p[1]] = 1
        deg[p[0]] = deg[p[0]] + 1
        deg[p[1]] = deg[p[1]] + 1
    n = max(deg.values())
    print(n)
    H = [0] * (n + 1)
    for v in deg.values():
        H[v] = H[v] + 1
    print(H[0])
    X = np.arange(st,n+1)
    if not breaks:
        breaks = len(X)
    _ = plt.hist(X, bins = breaks, weights=H[st:n+1])
    if output: plt.savefig(fname=output)


#Draw Weight Histogram
def get_weight_spectra(adj_L,breaks = 10, rang=None, log = False, wtype='plus', output = None):
    n = len(adj_L)
    spec = [0] * (n+n+1)
    X = [0] + [1/(i+1) for i in range(2*n)]
    #X = np.arange(0, 2*n+1)
    for i in range(n):
        for j in adj_L[i]:
            w = len(adj_L[i]) + len(adj_L[j]) if wtype == 'plus' else max(len(adj_L[i]), len(adj_L[j]))
            if i<j: spec[w] += 1
    
    _ = plt.hist(X, bins = breaks, range = rang, log = log, weights=spec)
    if output: plt.savefig(output)      

In [ ]:
#Fractal Dimension
def json_decode(fname = './'):
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        return json_data


path = './AdjList'
for dirname in os.listdir(path):
    if dirname[0] == '.':
        continue
    path_spec = os.path.join(path, dirname)
    #path_export_spec = os.path.join(path_export, dirname)
    for filename in os.listdir(path_spec):
        if filename.split('.')[1] != 'txt':
            continue
        #filename_export = os.path.join(path_export_spec, filename)
        filename_spec = os.path.join(path_spec, filename)
        subprocess.call(['../graph-sketch-fractality-master/bin/box_cover','-type=auto','-graph='+filename_spec,'-method=sketch','-random_seed=1124747'])


def jlog_linregress(fname = None, output=None):
    json_data = json_decode(fname)
    
    size = np.array(json_data['size'])
    rad = np.array(json_data['radius'])
    slope, intercept, r_value, p_value, std_err = linregress(np.log10(rad), np.log10(size))
    
    plt.plot(np.log10(rad), np.log10(size), 'k.', label='_nolegend_')
    plt.plot(np.log10(rad), np.log10(rad)*slope+intercept, label='y=''{:.3f}'.format(slope)+'x+''{:.3f}'.format(intercept))
    plt.title(json_data['run']['args'][2].split('/')[-1].split('.')[0])
    plt.xlabel('Log(rad)')
    plt.ylabel('Log(size)')
    for i in range(len(rad)):
        plt.annotate(size[i], (np.log10(rad)[i], np.log10(size)[i]))
    plt.legend()
    fig = plt.gcf()
    plt.show()
    if output:
        path_lin = ''.join(np.delete('/'.join(np.delete(json_data['run']['args'][2].split('/'),[0,1])).split('.'),-1))
        path_lin = './Fractal/'+path_lin+'.png'
        #print(path_lin)
        fig.savefig(path_lin)
    return slope


path = './jlog'
for name in os.listdir(path):
    if 'jlog' not in name: continue
    full_name = os.path.join(path, name)
    s = jlog_linregress(full_name, output=True)
    print(s)